In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

# Import the API key.
from config import weather_api_key

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

272

In [5]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [24]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | provideniya
Processing Record 2 of Set 1 | ushuaia
Processing Record 3 of Set 1 | ulundi
Processing Record 4 of Set 1 | keti bandar
Processing Record 5 of Set 1 | vaini
Processing Record 6 of Set 1 | punta arenas
Processing Record 7 of Set 1 | kapaa
Processing Record 8 of Set 1 | iqaluit
Processing Record 9 of Set 1 | airai
Processing Record 10 of Set 1 | chokurdakh
Processing Record 11 of Set 1 | pangnirtung
Processing Record 12 of Set 1 | zopilotepe
Processing Record 13 of Set 1 | quatre cocos
Processing Record 14 of Set 1 | francia
Processing Record 15 of Set 1 | san patricio
Processing Record 16 of Set 1 | goteborg
City not found. Skipping...
Processing Record 17 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 18 of Set 1 | anadyr
Processing Record 19 of Set 1 | rikitea
Processing Record 20 of Set 1 | high prairie
Processing Record 21 of Set 1 | namibe
Processing Record 22 of Set 1 | san quintin
Processing Record 23 of Set 1 | sal

In [25]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Provideniya,RU,64.3833,-173.3000,-5.44,85,10,6.22,clear sky
1,Ushuaia,AR,-54.8000,-68.3000,55.06,41,0,0.00,clear sky
2,Ulundi,ZA,-28.3352,31.4162,60.12,87,71,6.35,broken clouds
3,Keti Bandar,PK,24.1447,67.4497,74.66,71,18,3.74,few clouds
4,Vaini,TO,-21.2000,-175.2000,80.22,77,100,17.11,overcast clouds
5,Punta Arenas,CL,-53.1500,-70.9167,59.11,38,75,14.97,broken clouds
6,Kapaa,US,22.0752,-159.3190,75.67,60,0,5.21,clear sky
7,Iqaluit,CA,63.7506,-68.5145,21.11,91,100,8.88,light snow
8,Airai,TL,-8.9266,125.4092,61.61,91,98,1.63,overcast clouds
9,Chokurdakh,RU,70.6333,147.9167,-34.42,100,80,1.99,broken clouds


In [26]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")